Daftar Scrapping
1. Review
2. Seat Type
3. Seat Comfort
4. Cabin Staff Service
5. Food & Beverages
6. Value For Money
7. Recommended

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd
from numpy import NaN
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
page_number = 15
pages_size = 100

seat_type = []
recommended = []
datetimes = []
text_reviews = []

# review star fill
seat_comfort_ratings = []
cabin_staff_service_ratings = []
food_beverages_ratings = []
inflight_entertainment_ratings = []
ground_service_ratings = []
wifi_connectivity_ratings = []

for i in range(1,page_number+1):
  # request ke url website
  url = f"https://www.airlinequality.com/airline-reviews/british-airways/page/{i}/?sortby=post_date%3ADesc&pagesize={pages_size}"
  response = requests.get(url)
  html = response.text

  # membuat objek BeautifulSoup
  soup = BeautifulSoup(html, "html.parser")

  # mengambil informasi seat type
  td_elements = soup.find_all("td", {"class": "review-value"})
  for td in td_elements:
    if "Seat Type" in td.previous_sibling.text:
      seat_type.append(td.text)

  # mengambil informasi recommend yes/no
  td_recommend = soup.find_all("td", {"class": "review-rating-header recommended"})
  for recommend in td_recommend:
    recommended.append(recommend.next_sibling.text)

  # mengambil tanggal tiap review
  date = soup.find_all('time', itemprop='datePublished')
  for dates in date:
    datetimes.append(dates['datetime'])

  # mengambil informasi text review
  for para in soup.find_all("div", {"class": "text_content"}):
    text_reviews.append(para.get_text())

  # mengambil informasi review starfill
  review_containers = soup.select('article[itemprop="review"]')
  for review in review_containers:
    # seat comfort
    seat_comfort = review.select_one('td.review-rating-header.seat_comfort')
    if seat_comfort is not None:
      seat_comfort_rating = seat_comfort.find_next_sibling('td').select('span.star.fill')[-1].text
      seat_comfort_ratings.append(seat_comfort_rating)
    else:
      # handle the case where seat_comfort is None
      seat_comfort_ratings.append(NaN)

    # cabin staff service
    cabin_staff_service = review.select_one('td.review-rating-header.cabin_staff_service')
    if cabin_staff_service is not None:
      cabin_staff_service_rating = cabin_staff_service.find_next_sibling('td').select('span.star.fill')[-1].text
      cabin_staff_service_ratings.append(cabin_staff_service_rating)
    else:
      # handle the case where food_beverages is None
      cabin_staff_service_ratings.append(NaN)

    # food beverages
    food_beverages = review.select_one('td.review-rating-header.food_and_beverages')
    if food_beverages is not None:
      food_beverages_rating = food_beverages.find_next_sibling('td').select('span.star.fill')[-1].text
      food_beverages_ratings.append(food_beverages_rating)
    else:
      # handle the case where food_beverages is None
      food_beverages_ratings.append(NaN)

    # ground_service
    ground_service = review.select_one('td.review-rating-header.ground_service')
    if ground_service is not None:
      ground_service_rating = ground_service.find_next_sibling('td').select('span.star.fill')[-1].text
      ground_service_ratings.append(ground_service_rating)
    else:
      # handle the case where food_beverages is None
      ground_service_ratings.append(NaN)

In [ ]:
# Membuat dataframe
df_scrap = pd.DataFrame()

# Memasukkan kolom dari list yang sudah dibuat dari hasil scrapping
df_scrap['reviews'] = text_reviews
df_scrap['date_reviews'] = datetimes
df_scrap['seat_type'] = seat_type
df_scrap['seat_comfort_ratings'] = seat_comfort_ratings
df_scrap['cabin_staff_service_ratings'] = cabin_staff_service_ratings
df_scrap['food_beverages_ratings'] = food_beverages_ratings
df_scrap['ground_service_ratings'] = ground_service_ratings
df_scrap['recommended'] = recommended

In [ ]:
df_scrap

,reviews,date_reviews,seat_type,seat_comfort_ratings,cabin_staff_service_ratings,food_beverages_ratings,ground_service_ratings,recommended
0,✅ Trip Verified | Easy check in a T5. Galleri...,2023-08-13,Business Class,4,5,5,4,yes
1,"Not Verified | Flight delayed by an hour, it ...",2023-08-12,Economy Class,1,1,NaN,1,no
2,Not Verified | The staff are very rude and not...,2023-08-11,Economy Class,1,1,1,1,no
3,✅ Trip Verified | Good domestic flight operat...,2023-08-08,Economy Class,4,4,3,5,yes
4,Not Verified | Failed at all basic travel fund...,2023-08-08,Economy Class,3,3,NaN,1,no
...,...,...,...,...,...,...,...,...
1495,✅ Verified Review | London to Zagreb. British...,2017-07-01,Economy Class,NaN,NaN,NaN,1,no
1496,✅ Verified Review | Manchester to Philadelphi...,2017-07-01,Economy Class,2,1,1,2,no
1497,✅ Verified Review | Just landed from the seco...,2017-06-30,First Class,5,5,5,5,yes
1498,✅ Verified Review | Frankfurt to London Heath...,2017-06-30,Economy Class,2,2,NaN,1,no


In [ ]:
df_scrap.to_csv('British Airways.csv', index=False)